In [ ]:
!pip install konlpy
!pip install transformers
!pip install sentencepiece

from konlpy.tag import Komoran, Okt
from tqdm import tqdm, trange
import sentencepiece as spm
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

In [ ]:
%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(515)

In [ ]:
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# preprocessing
def load_vocab(file):
    vocab = spm.SentencePieceProcessor()
    vocab.load(file)
    return vocab

In [ ]:
# utils
def train(model, criterion, optimizer, input_size, output_size,
          train_dataloader, validation_dataloader,
          nb_epochs):
    
    # Train loop
    trn_loss_list = []
    val_loss_list = []
    for epoch in range(nb_epochs):

        # Train
        trn_loss = 0.0
        for trn_batch, train_samples in enumerate(train_dataloader):

            # train data setting
            x_train, y_train = train_samples
            x_train = x_train.unsqueeze(0).to(device)
            y_train = y_train.float().to(device)

            # train
            model.train()
            optimizer.zero_grad()
            hypothesis = model(x_train).view(-1, output_size)
            train_loss = criterion(hypothesis, y_train)
            train_loss.backward()
            optimizer.step()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm)

            # train loss
            trn_loss += train_loss.item() / len(train_dataloader)
        trn_loss_list.append(trn_loss)

        # Evaluation
        with torch.no_grad():
            val_loss = 0.0
            for val_batch, validation_samples in enumerate(validation_dataloader):

                # validatoin data setting
                x_validation, y_validation = validation_samples
                x_validation = x_validation.unsqueeze(0).to(device)
                y_validation = y_validation.float().to(device)

                # evaluation
                model.eval()
                prediction = model(x_validation).view(-1, output_size)
                validation_loss = criterion(prediction, y_validation)

                # validation loss
                val_loss += validation_loss.item() / len(validation_dataloader)
            val_loss_list.append(val_loss)

        print("Epoch: {:3d} | Train Loss: {:.6f} | Val Loss: {:.6f}".format(epoch + 1, trn_loss, val_loss))
    #torch.save(model, './data/temperature_model.pt')
    return trn_loss_list, val_loss_list

def checkdata(dataloader):
    for batch, values in enumerate(dataloader):
        while batch < 6:
            label, enc_inputs, dec_inputs = values
            print("{} Batch".format(batch))
            print("Input: {}".format(enc_inputs.size()))
            print("Target: {}".format(dec_inputs.size()))
            break
    return label, enc_inputs, dec_inputs

def checkfunction(criterion, dataloader, output_size, epoch):
    for batch, value in enumerate(dataloader):
        x_, y_ = value
        x_ = x_.unsqueeze(0).to(device)
        y_ = y_.float().to(device)
        hypothesis = model(x_).view(-1, output_size)
        loss = criterion(hypothesis, y_)
        print("Batch:", batch+1)
        print("Input size:", x_.size())
        print("Target size:", y_.size())
        print("Hypothesis", hypothesis)
        print("Hypothesis size:", hypothesis.size())
        print("Loss:", loss)
        print()
        if batch == epoch:
            break

In [ ]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

class CustomDataSet(data.Dataset):
    def __init__(self, vocab, infile):

        self.vocab = vocab
        self.labels = []
        self.sentences = []

        line_cnt = 0
        with open(infile, "r") as f:
            for line in f:
                line_cnt += 1

        with open(infile, "r") as f:
            for i, line in enumerate(tqdm(f, total=line_cnt, desc=f"Loading {infile}", unit=" lines")):
                data = json.loads(line)
                self.labels.append(data["label"])
                self.sentences.append([vocab.piece_to_id(p) for p in data["doc"]])
    
    def __len__(self):
        assert len(self.labels) == len(self.sentences)
        return len(self.labels)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor([self.vocab.piece_to_id("[BOS]")]))
        
def movie_collate_fn(inputs):
    labels, enc_inputs, dec_inputs = list(zip(*inputs))

    enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True, padding_value=0)
    dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels, dim=0),
        enc_inputs,
        dec_inputs,
    ]
    return batch

In [ ]:
# Model
class Classification(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        # values
        self.config = config

        # layers
        self.transformer = Transformer(self.config)
        self.projection = nn.Linear(self.config['hidden_size'], self.config['output_size'], bias = False)

    def forward(self, enc_inputs, dec_inputs):
        dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs = self.transformer(enc_inputs, dec_inputs)
        dec_outputs, _ = torch.max(dec_outputs, dim=1)
        logits = self.projection(dec_outputs)
        return logits, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

    def save(self, epoch, loss, score, path):
        torch.save({
            "epoch": epoch,
            "loss": loss,
            "score": score,
            "state_dict": self.state_dict()
        }, path)
    
    def load(self, path):
        save = torch.load(path)
        self.load_state_dict(save["state_dict"])
        return save["epoch"], save["loss"], save["score"]

class Transformer(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config

        # layers
        self.encoder = Encoder(self.config)
        self.decoder = Decoder(self.config)

    def forward(self, enc_inputs, dec_inputs):
        enc_outputs, enc_self_attn_probs = self.encoder(enc_inputs)
        dec_outputs, dec_self_attn_probs, dec_enc_attn_probs = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        return dec_outputs, enc_self_attn_probs, dec_self_attn_probs, dec_enc_attn_probs

In [ ]:
# Encoding
class Encoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config

        # layers
        self.enc_emb = nn.Embedding(self.config['num_encoding_vocab'], self.config['hidden_size'])
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config['num_encoding_sequence'] + 1, self.config['hidden_size']))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze = True)
        self.layers = nn.ModuleList([EncoderLayer(self.config) for _ in range(self.config['num_layers'])])
  
    def forward(self, x):
        positions = torch.arange(x.size(1), device=x.device, dtype=x.dtype).expand(x.size(0), x.size(1)).contiguous() + 1
        pos_mask = x.eq(self.config['i_pad'])
        outputs = self.enc_emb(x) + self.pos_emb(positions)
        attn_mask = get_attn_pad_mask(x, x, self.config['i_pad'])

        attn_probs = []
        for layer in self.layers:
            outputs, attn_prob = layer(outputs, attn_mask)
            attn_probs.append(attn_prob)
        return outputs, attn_probs

class EncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config

        # layers
        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config['hidden_size'], eps=self.config['layer_norm_epsilon'])
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config['hidden_size'], eps=self.config['layer_norm_epsilon'])

    def forward(self, x, attn_mask):
        att_outputs, attn_prob = self.self_attn(x, x, x, attn_mask)
        att_outputs = self.layer_norm1(x + att_outputs)

        ffn_outputs = self.pos_ffn(att_outputs)
        ffn_outputs = self.layer_norm2(ffn_outputs + att_outputs)
        return ffn_outputs, attn_prob

class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config
        
        # layers
        self.W_Q = nn.Linear(self.config['hidden_size'], self.config['n_head'] * self.config['d_head'])
        self.W_K = nn.Linear(self.config['hidden_size'], self.config['n_head'] * self.config['d_head'])
        self.W_V = nn.Linear(self.config['hidden_size'], self.config['n_head'] * self.config['d_head'])
        self.scaled_dot_attn = ScaledDotProductAttention(self.config)
        self.linear = nn.Linear(self.config['n_head'] * self.config['d_head'], self.config['hidden_size'])
        self.dropout = nn.Dropout(self.config['dropout'])

    def forward(self, Q, K, V, attn_mask):
        batch_size = Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, self.config['n_head'], self.config['d_head']).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.config['n_head'], self.config['d_head']).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.config['n_head'], self.config['d_head']).transpose(1,2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.config['n_head'], 1, 1)

        context, attn_prob = self.scaled_dot_attn(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.config['n_head'] * self.config['d_head'])
        output = self.linear(context)
        output = self.dropout(output)
        return output, attn_prob

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        # values
        self.config = config

        # layers
        self.conv1 = nn.Conv1d(in_channels=self.config['hidden_size'], out_channels=self.config['d_ff'], kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=self.config['d_ff'], out_channels=self.config['hidden_size'], kernel_size=1)
        self.active = F.gelu
        self.dropout = nn.Dropout(self.config['dropout'])

    def forward(self, x):
        output = self.active(self.conv1(x.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        output = self.dropout(output)
        return output

class ScaledDotProductAttention(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config
        self.dropout = nn.Dropout(self.config['dropout'])
        self.scale = 1 / (self.config['d_head'] ** 0.5)

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)).mul_(self.scale)
        scores.masked_fill_(attn_mask, -1e9)
        attn_prob = nn.Softmax(dim=-1)(scores)
        attn_prob = self.dropout(attn_prob)
        context = torch.matmul(attn_prob, V)
        return context, attn_prob

In [ ]:
# Decoding
class Decoder(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config

        # layers
        self.dec_embs = nn.Embedding(self.config['num_decoding_vocab'], self.config['hidden_size'])
        sinusoid_table = torch.FloatTensor(get_sinusoid_encoding_table(self.config['num_decoding_sequence'] + 1, self.config['hidden_size']))
        self.pos_emb = nn.Embedding.from_pretrained(sinusoid_table, freeze=True)
        self.layers = nn.ModuleList([DecoderLayer(self.config) for _ in range(self.config['num_layers'])])

    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        positions = torch.arange(dec_inputs.size(1), device=dec_inputs.device, dtype=dec_inputs.dtype).expand(dec_inputs.size(0), dec_inputs.size(1)).contiguous() + 1
        pos_mask = dec_inputs.eq(self.config['i_pad'])
        positions.masked_fill_(pos_mask, 0)
    
        dec_outputs = self.dec_embs(dec_inputs) + self.pos_emb(positions)

        dec_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs, self.config['i_pad'])
        dec_attn_decoder_mask = get_attn_decoder_mask(dec_inputs)
        dec_self_attn_mask = torch.gt((dec_attn_pad_mask + dec_attn_decoder_mask), 0)
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs, self.config['i_pad'])

        self_attn_probs, dec_enc_attn_probs = [], []
        for layer in self.layers:
            dec_outputs, self_attn_prob, dec_enc_attn_prob = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            self_attn_probs.append(self_attn_prob)
            dec_enc_attn_probs.append(dec_enc_attn_prob)
        return dec_outputs, self_attn_probs, dec_enc_attn_probs

class DecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()

        # values
        self.config = config

        # layers
        self.self_attn = MultiHeadAttention(self.config)
        self.layer_norm1 = nn.LayerNorm(self.config['hidden_size'], eps=self.config['layer_norm_epsilon'])
        self.dec_enc_attn = MultiHeadAttention(self.config)
        self.layer_norm2 = nn.LayerNorm(self.config['hidden_size'], eps=self.config['layer_norm_epsilon'])
        self.pos_ffn = PoswiseFeedForwardNet(self.config)
        self.layer_norm3 = nn.LayerNorm(self.config['hidden_size'], eps=self.config['layer_norm_epsilon'])
    
    def forward(self, dec_inputs, enc_outputs, self_attn_mask, dec_enc_attn_mask):
        self_att_outputs, self_attn_prob = self.self_attn(dec_inputs, dec_inputs, dec_inputs, self_attn_mask)
        self_att_outputs = self.layer_norm1(dec_inputs + self_att_outputs)
        dec_enc_att_outputs, dec_enc_attn_prob = self.dec_enc_attn(self_att_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_enc_att_outputs = self.layer_norm2(self_att_outputs + dec_enc_att_outputs)
        ffn_outputs = self.pos_ffn(dec_enc_att_outputs)
        ffn_outputs = self.layer_norm3(dec_enc_att_outputs + ffn_outputs)
        return ffn_outputs, self_attn_prob, dec_enc_attn_prob

In [ ]:
# Encoding/Decoding util functions
def get_sinusoid_encoding_table(n_seq, hidden_size):
    def cal_angle(position, i_hidn):
        return position / np.power(10000, 2 * (i_hidn // 2) / hidden_size)
    def get_posi_angle_vec(position):
        return [cal_angle(position, i_hidn) for i_hidn in range(hidden_size)]

    sinusoid_table = np.array([get_posi_angle_vec(i_seq) for i_seq in range(n_seq)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # even index sin 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # odd index cos
    return sinusoid_table

def get_attn_pad_mask(seq_q, seq_k, i_pad):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    pad_attn_mask = seq_k.eq(i_pad).unsqueeze(1).expand(batch_size, len_q, len_k)
    return pad_attn_mask

def get_attn_decoder_mask(seq):
    subsequent_mask = torch.ones_like(seq).unsqueeze(-1).expand(seq.size(0), seq.size(1), seq.size(1))
    subsequent_mask = subsequent_mask.triu(diagonal=1) # upper triangular part of a matrix(2-D)
    return subsequent_mask

In [ ]:
vocab = load_vocab("/content/drive/My Drive/data/kowiki.model")

In [ ]:
config = Config({
    "batch_size": 64, 
    "num_encoding_vocab": len(vocab),
    "num_decoding_vocab": len(vocab),
    "num_sequence": 64, 
    "num_encoding_sequence": 512,
    "num_decoding_sequence": 512,
    "num_layers": 12,
    "hidden_size": 128,
    "i_pad": 0,
    "d_ff": 3072,
    "n_head": 12,
    "d_head": 64,
    "dropout": 0.1,
    "layer_norm_epsilon": 1e-12,
    "output_size": 2,
    "weight_decay": 0,
    "learning_rate": 5e-5,
    "max_norm": 5,
    "adam_epsilon": 1e-8,
    "warmup_steps": 0,
    "nb_epochs": 100
})

In [ ]:
trn_dataset = CustomDataSet(vocab, "/content/drive/My Drive/data/ratings_train.json")
trn_dataloader = data.DataLoader(trn_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=movie_collate_fn)
test_dataset = CustomDataSet(vocab, "/content/drive/My Drive/data/ratings_test.json")
test_dataloader = data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=movie_collate_fn)

Loading /content/drive/My Drive/data/ratings_train.json: 100%|██████████| 149995/149995 [00:05<00:00, 26452.55 lines/s]
Loading /content/drive/My Drive/data/ratings_test.json: 100%|██████████| 49997/49997 [00:02<00:00, 24821.27 lines/s]


In [ ]:
trn_label, trn_enc_inputs, trn_dec_inputs = checkdata(trn_dataloader)
test_label, test_enc_inputs, test_dec_inputs = checkdata(test_dataloader)

0 Batch
Input: torch.Size([64, 88])
Target: torch.Size([64, 1])
1 Batch
Input: torch.Size([64, 100])
Target: torch.Size([64, 1])
2 Batch
Input: torch.Size([64, 92])
Target: torch.Size([64, 1])
3 Batch
Input: torch.Size([64, 79])
Target: torch.Size([64, 1])
4 Batch
Input: torch.Size([64, 97])
Target: torch.Size([64, 1])
5 Batch
Input: torch.Size([64, 87])
Target: torch.Size([64, 1])
0 Batch
Input: torch.Size([64, 97])
Target: torch.Size([64, 1])
1 Batch
Input: torch.Size([64, 95])
Target: torch.Size([64, 1])
2 Batch
Input: torch.Size([64, 86])
Target: torch.Size([64, 1])
3 Batch
Input: torch.Size([64, 58])
Target: torch.Size([64, 1])
4 Batch
Input: torch.Size([64, 84])
Target: torch.Size([64, 1])
5 Batch
Input: torch.Size([64, 84])
Target: torch.Size([64, 1])


In [ ]:
print("Train label:", trn_label[0])
print("Train encoder input:", trn_enc_inputs[0])
print("Train decoder input:", trn_dec_inputs[0])
print("Test label:", test_label[0])
print("Test encoder input:", test_enc_inputs[0])
print("Test decoder input:", test_dec_inputs[0])

Train label: tensor(1)
Train encoder input: tensor([1225,   23, 3589, 3778, 3931, 3760, 1403, 3835, 3596, 3857, 3760, 3587,
        7838,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0])
Train decoder input: tensor([2])
Test label: tensor(0)
Test encoder input: tensor([3587, 5053, 3953, 3640, 1803, 2711,  339,   48,  818,  952, 1598,  121,
        3605, 3768, 3587, 5053, 3953, 3591,  226,    9,  212, 3611, 3768,  115,
          26, 3794, 3750,  748,   26, 4973, 3602, 1920, 1920, 3686, 3877, 3804,
        4014, 2039, 3699, 3996, 4502,  228, 3596, 4282

In [ ]:
encoder = Encoder(config)
decoder = Decoder(config)

trn_enc_outputs, trn_enc_attn_probs = encoder(trn_enc_inputs)
trn_dec_outputs, trn_self_attn_probs, trn_dec_enc_attn_probs = decoder(trn_dec_inputs, trn_enc_inputs, trn_enc_outputs)

In [ ]:
print(encoder)
print(decoder)

Encoder(
  (enc_emb): Embedding(8007, 128)
  (pos_emb): Embedding(513, 128)
  (layers): ModuleList(
    (0): EncoderLayer(
      (self_attn): MultiHeadAttention(
        (W_Q): Linear(in_features=128, out_features=768, bias=True)
        (W_K): Linear(in_features=128, out_features=768, bias=True)
        (W_V): Linear(in_features=128, out_features=768, bias=True)
        (scaled_dot_attn): ScaledDotProductAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (linear): Linear(in_features=768, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm1): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (pos_ffn): PoswiseFeedForwardNet(
        (conv1): Conv1d(128, 3072, kernel_size=(1,), stride=(1,))
        (conv2): Conv1d(3072, 128, kernel_size=(1,), stride=(1,))
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (layer_norm2): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    )
    (1

In [ ]:
print(trn_enc_outputs[0])
print(trn_enc_attn_probs[0][0][0])
print(trn_dec_outputs[0])
print(trn_self_attn_probs[0][0][0])
print(trn_dec_enc_attn_probs[0][0][0])

tensor([[ 0.3312, -1.0019, -0.7045,  ..., -0.4869, -0.2089, -0.0589],
        [ 0.1483, -1.0929,  0.1269,  ...,  1.4843, -1.2941,  0.1250],
        [-2.4093, -1.1495, -1.2197,  ...,  0.6273, -0.0952,  0.6801],
        ...,
        [ 0.1422,  0.2337, -0.9410,  ...,  0.8403, -0.4742, -0.4359],
        [-0.3164,  0.5324, -0.6484,  ...,  0.7306, -0.7952, -0.1375],
        [-0.5716,  0.7938, -1.1231,  ...,  1.0023, -0.3596, -0.2672]],
       grad_fn=<SelectBackward>)
tensor([[0.2411, 0.0468, 0.1369,  ..., 0.0000, 0.0000, 0.0000],
        [0.1681, 0.0570, 0.2088,  ..., 0.0000, 0.0000, 0.0000],
        [0.1639, 0.0919, 0.1126,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0694, 0.0772, 0.0676,  ..., 0.0000, 0.0000, 0.0000],
        [0.0653, 0.0762, 0.0677,  ..., 0.0000, 0.0000, 0.0000],
        [0.0673, 0.0748, 0.0706,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<SelectBackward>)
tensor([[ 8.2483e-01, -1.6040e-01, -2.2424e-01,  4.3319e-02,  3.0983e-01,
          3.1724e-01, -7.393

In [ ]:
print(np.shape(trn_enc_outputs))
print(np.shape(trn_enc_attn_probs))
print(np.shape(trn_dec_outputs))
print(np.shape(trn_self_attn_probs))
print(np.shape(trn_dec_enc_attn_probs))

torch.Size([64, 87, 128])
(12,)
torch.Size([64, 1, 128])
(12,)
(12,)


In [ ]:
model = Classification(config).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr = config['learning_rate'])

In [ ]:
hypothesis = model(trn_enc_inputs, trn_dec_inputs)

In [ ]:
hypothesis[0]

tensor([[ 0.6885, -1.1506],
        [ 0.9400,  0.3825],
        [ 0.8728,  0.6479],
        [ 0.8540, -0.0659],
        [ 0.5589,  0.2431],
        [ 1.1899, -0.3123],
        [ 0.6480, -0.0479],
        [ 1.1072,  0.3918],
        [ 1.3039, -0.1402],
        [ 0.9356,  0.3019],
        [ 1.1641,  0.4604],
        [ 1.4840, -0.7345],
        [ 1.2879, -0.2980],
        [ 0.6916,  0.8984],
        [ 0.8344,  0.1325],
        [ 1.1648,  0.1282],
        [ 0.7323, -0.1503],
        [ 1.0404,  0.5493],
        [ 0.9016, -0.1578],
        [ 0.7259, -0.5757],
        [ 0.6604, -0.0790],
        [ 0.6236,  0.0972],
        [ 1.2589,  0.3647],
        [ 0.4845, -0.1122],
        [ 0.6667, -0.0321],
        [ 1.0663, -0.4645],
        [ 0.4871, -0.1381],
        [ 0.2773,  0.5033],
        [ 0.3979, -0.3347],
        [ 1.1419, -0.4560],
        [ 0.5753, -0.7924],
        [ 0.8543,  0.7123],
        [ 1.5589, -0.1648],
        [ 1.0534,  0.3961],
        [ 0.9457,  0.2923],
        [ 0.9636,  0

In [ ]:
#print("Model:", model)
print("Criterion:", criterion)
print("Optimizer:", optimizer)
print(checkfunction(criterion = criterion, 
                    dataloader = trn_dataloader, 
                    output_size = config['output_size'], epoch = 1))

In [ ]:
trn_loss, val_loss = train(model, criterion, optimizer,input_size, output_size,
                           trn_dataloader, val_dataloader, nb_epochs)